In [ ]:
import matplotlib.pyplot as plt
from numpy import pi as π
import numpy as np
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import dframe_tools as dtools
# dtools.help()

import plot_tools as ptools
# ptools.help()

import gen_resubmit as gensub
# gensub.help()

path = '/home/sang/UVM/He4vacuum/OUTPUT'
dbase = dtools.df_pimc(path, skip = 0, noenergy = 1, replace = 1)

dfdict = ptools.datadic(dbase)

In [ ]:
def keysorting(k):
    t = k.split(',')
    n = float(t[0])*100000 + float(t[1])
    return n

keylst = []
for keys in dfdict.keys():
    keylst.append(keys)
keylst = sorted(keylst, key = lambda k: keysorting(k))
print(keysorting(keylst[0]))
print(keylst)
print('number of category:', len(keylst))
for keys in keylst:
    print(ptools.nparallel( dfdict[keys] ), end=' ')

In [ ]:
# Find mu for simulations with 'relaxmu' option
realmu = {}
subset = dbase
for i in range(len(subset)):
    file_path = subset.iloc[i]['filedic']['log']
    lid = subset.iloc[i]['id']
#     print(file_path)
    ref = file_path.find('log')
#     print(file_path[ref+19:ref+27])
    realmu[lid] = file_path[ref+19:ref+27]

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
def sort_list(list1, list2):
 
    zipped_pairs = zip(list2, list1)
 
    z = [x for _, x in sorted(zipped_pairs)]
     
    return z

def linear_fit(subset):
    mu = []
    muave = []
    n = []
    nave = []
    nerr = []
    for i in range(len(subset)):
        if subset.iloc[i]['n'] < 0.9 and subset.iloc[i]['n'] > 0.01:
            lid = subset.iloc[i]['id']
            muave.append(float(realmu[lid]))
            nave.append(subset.iloc[i]['n'])
            nerr.append(subset.iloc[i]['nerr'])
        elif subset.iloc[i]['n'] < 4 and subset.iloc[i]['n'] >= 0.9:
            lid = subset.iloc[i]['id']
            mu.append(float(realmu[lid]))
            n.append(subset.iloc[i]['n'])
            nerr.append(subset.iloc[i]['nerr'])
    if len(muave)>0:
        mu.append(sum(muave)/len(muave))
        n.append(sum(nave)/len(nave))
    
#     a1,a1_err = get_a(mu,n,nerr)
#     print(a1)
    n = sort_list(n,mu)
    mu = sorted(mu)
    
    mu = np.array(mu)
    n = np.array(n)
    
    if len(mu) >= 2:

        reg = LinearRegression().fit(mu.reshape((-1, 1)), n)
        # print(reg.score(x, y))
        
#         print(reg.coef_)
#         print(reg.intercept_)

        fx = np.linspace(min(mu)*1.1,max(mu)*0.9,20)
    #     plt.plot(fx,a1[0]+a1[1]*fx, label = 'a method')
        plt.scatter(mu, n)
        plt.plot(fx,reg.intercept_+reg.coef_*fx, label = 'Reg method')
#         print(-reg.intercept_/reg.coef_)
        return -reg.intercept_/reg.coef_[0], nerr
    else:
        return mu[0], nerr

In [ ]:
# New result
# This print strain, total adsorption sites, real chemical potential, maximum error
for s in range(31):
    for totN in (16,64):
        strain = 0.0 + 0.01*s

        lab = f'{strain}, {totN}'
        subset = dbase[(dbase['strain'] == strain)&(dbase['totN'] == totN)]

        intercept, nerr = linear_fit(subset)

        print(lab+',', intercept, max(nerr))
        vacuums[lab] = intercept

In [ ]:
# Rough data
plotx1 = []
ploty1 = []
for s in range(31):
    for totN in (16,):
        strain = 0.0 + 0.01*s
        lab = f'{strain}, {totN}'
        plotx1.append(strain*100)
        ploty1.append(vacuums[lab])
plotx2 = []
ploty2 = []
for s in range(31):
    for totN in (64,):
        strain = 0.0 + 0.01*s
        lab = f'{strain}, {totN}'
        plotx2.append(strain*100)
        ploty2.append(vacuums[lab])
plt.plot(plotx1, ploty1)
plt.plot(plotx2, ploty2)
plt.show()

In [ ]:
# UnivariateSpline Curve
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline
from scipy.interpolate import UnivariateSpline
new_x = np.linspace(0,30,100)
f1 = UnivariateSpline(plotx, ploty,)
f2 = UnivariateSpline(plotx, plotydiff,)

dev = sum(f2(new_x))/len(f2(new_x))
print(dev, max(f2(new_x)))
plt.fill_between(new_x, f1(new_x) - f2(new_x) -0.5, f1(new_x) + f2(new_x) +0.5,
                 color='gray', alpha=0.2)

In [ ]:
# Export data
# np.savetxt("/home/sang/UVM/He4vacuum/vacuumline_x.csv", new_x, delimiter =", ", fmt='%.12f')
# np.savetxt("/home/sang/UVM/He4vacuum/vacuumline_yd.csv", f1(new_x) - f2(new_x), delimiter =", ", fmt='%.12f')
# np.savetxt("/home/sang/UVM/He4vacuum/vacuumline_yu.csv", f1(new_x) + f2(new_x), delimiter =", ", fmt='%.12f')